Method 1 :

In [1]:
import librosa
import soundfile as sf

In [2]:
# 1. Load the original audio
input_file = "sawasdee_hooyang_3.wav"
y, sr = librosa.load(input_file, sr=None)  # sr=None keeps original sample rate


In [3]:
# 2. Apply pitch shifting (e.g., +5 semitones → more childlike)
y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=2)


In [4]:
# 3. Save the result to a new file
output_file = "child_voice.wav"
sf.write(output_file, y_shifted, sr)

print(f"Saved modified audio to {output_file}")

Saved modified audio to child_voice.wav


Method 2 

In [5]:
import numpy as np
import librosa
import soundfile as sf
from scipy.signal import butter, filtfilt

try:
    import noisereduce as nr
    HAS_NR = True
except Exception:
    HAS_NR = False

def butter_highpass(y, sr, cutoff=120.0, order=4):
    nyq = 0.5 * sr
    b, a = butter(order, cutoff/nyq, btype='high', analog=False)
    return filtfilt(b, a, y)

input_path  = "sawasdee_hooyang_3.wav"
output_path = "child_voice_clean.wav"

# 1) Load
y, sr = librosa.load(input_path, sr=None, mono=True)

# 2) ลองตัดความถี่ต่ำ (ช่วยลดความอึด/ก้องบางส่วน)
y = butter_highpass(y, sr, cutoff=120)

# 3) (ออปชัน) ลดฮอลล์/ก้องพื้นฐานด้วย spectral gating
#    ถ้าไม่มี noisereduce โค้ดนี้จะข้ามไป
if HAS_NR:
    # ใช้ช่วงต้นไฟล์เป็น noise profile (ถ้ามีช่วงเงียบ ~0.5s)
    noise_profile = y[: int(0.5 * sr)] if len(y) > int(0.5 * sr) else y[: int(0.2 * sr)]
    y = nr.reduce_noise(y=y, y_noise=noise_profile, sr=sr, stationary=False, prop_decrease=0.9)

# 4) Pitch shift (ใช้รูปแบบ argument ใหม่ของ librosa)
#    ลอง n_steps = +4 ถึง +6; ยิ่งเยอะยิ่งเสี่ยงเกิดอาร์ติแฟ็กต์
y_ps = librosa.effects.pitch_shift(y, sr=sr, n_steps=5)

# 5) (ออปชัน) time-stretch เล็กน้อยให้พูดเร็วขึ้นเล็กน้อย (เสียงเด็กมักพูดเร็ว)
#    ค่า >1 คือเร็วขึ้น; 1.03–1.08 แนะนำเริ่มที่ 1.05
y_ps = librosa.effects.time_stretch(y_ps, rate=1.05)

# 6) ป้องกัน clipping และบันทึก
peak = np.max(np.abs(y_ps))
if peak > 0:
    y_ps = 0.98 * y_ps / peak

sf.write(output_path, y_ps, sr)
print(f"Saved: {output_path}")

Saved: child_voice_clean.wav


In [6]:
# pip install pyrubberband
import pyrubberband as prb
import soundfile as sf
import numpy as np

y, sr = librosa.load("sawasdee_hooyang_3.wav", sr=None, mono=True)

# ลดฮอลล์/ทำความสะอาดแบบเดียวกับด้านบนก่อน (optional) ...

# Pitch shift + preserve formants (ถ้าระบบรองรับ)
# semitone = +5
y_ps = prb.pitch_shift(y, sr, n_steps=5, rbargs={"formant": True, "crispness": 5})

# (ออปชัน) เร่งนิดหน่อย
y_ps = prb.time_stretch(y_ps, sr, 1.05)

# Normalize แล้วบันทึก
peak = np.max(np.abs(y_ps))
if peak > 0:
    y_ps = 0.98 * y_ps / peak
sf.write("child_voice_rubberband.wav", y_ps, sr)

RuntimeError: Failed to execute rubberband. Please verify that rubberband-cli is installed.

In [ ]:
# https://chatgpt.com/c/6901fd98-1a30-8323-8fb3-2f65a5a62f88